# 과대 낙폭주 관심종목 스크리닝 (Pullback Stock Screening)

이 노트북은 과대 낙폭을 경험한 중/소형주를 스크리닝하여 3가지 관심종목 리스트를 생성합니다.

## 스크리닝 기준

### 관심 1: 30일 과대 낙폭주 (신용가능)
- **낙폭 기준**: 30일 거래일 중 고가 대비 현재 종가 -70% 이하
- **종목 크기**: 중형주, 소형주
- **신용 거래**: 신용가능 종목만
- **정렬**: HC + LC 오름차순 (가장 낮은 값부터)

### 관심 2: 250일 과대 낙폭주 (신용가능)
- **낙폭 기준**: 250일 거래일 중 고가 대비 현재 종가 -70% 이하
- **종목 크기**: 중형주, 소형주
- **신용 거래**: 신용가능 종목만
- **정렬**: HC + LC 오름차순

### 관심 3: 250일 과대 낙폭주 (신용불가능)
- **낙폭 기준**: 250일 거래일 중 고가 대비 현재 종가 -70% 이하
- **종목 크기**: 중형주, 소형주
- **신용 거래**: 신용불가능 종목만
- **정렬**: HC + LC 오름차순

## 지표 설명

- **HC (High vs Current)**: 기간 내 고가 대비 현재가 등락률 (%)
  - 공식: `(현재가 - 고가) / 고가 * 100`
  - 예: -70% = 고가 대비 70% 하락
  
- **LC (Low vs Current)**: 기간 내 저가 대비 현재가 등락률 (%)
  - 공식: `(현재가 - 저가) / 저가 * 100`
  - 예: 10% = 저가 대비 10% 상승
  
- **HC + LC**: HC와 LC의 합계
  - 값이 낮을수록 고점에서 많이 떨어지고 저점에 가까움
  - 정렬 기준으로 사용

## 데이터 소스

- **daily_charts**: 일봉 OHLCV 데이터
- **stock_metadata**: 종목 메타데이터 (company_size, stock_state)


In [10]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import matplotlib.font_manager as fm
from matplotlib.font_manager import FontProperties
import warnings

# Suppress warnings
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')

# ============================================================================
# 스크리닝 설정 변수 (여기를 수정하세요)
# ============================================================================

# 낙폭 기준 (%)
PULLBACK_THRESHOLD_30D = 70   # 30일 기준 낙폭
PULLBACK_THRESHOLD_250D = 70  # 250일 기준 낙폭

# 종목 크기 필터
COMPANY_SIZES = ['중형주', '소형주']  # '대형주'를 추가하려면 리스트에 추가

# 조회 기간 (거래일 수)
LOOKBACK_DAYS_SHORT = 30    # 단기 기간
LOOKBACK_DAYS_LONG = 250    # 장기 기간

# ============================================================================

# Configure matplotlib for Korean font support
def setup_korean_font():
    """Setup matplotlib to display Korean characters properly"""
    korean_fonts = [
        'AppleGothic',       # macOS
        'Malgun Gothic',     # Windows
        'NanumGothic',       # Cross-platform
        'Nanum Gothic',
        'NanumBarunGothic',
        'UnDotum',           # Linux
    ]
    
    available_fonts = [f.name for f in fm.fontManager.ttflist]
    for font in korean_fonts:
        if font in available_fonts:
            plt.rc('font', family=font)
            plt.rc('axes', unicode_minus=False)
            print(f"✓ Korean font set to: {font}")
            return True
    
    plt.rcParams['font.family'] = 'DejaVu Sans'
    plt.rcParams['axes.unicode_minus'] = False
    print("⚠ No Korean font found, using fallback.")
    return False

def get_korean_font():
    """Get Korean font properties for explicit text rendering"""
    korean_fonts = [
        'AppleGothic', 'Malgun Gothic', 'NanumGothic',
        'Nanum Gothic', 'NanumBarunGothic', 'UnDotum'
    ]
    
    available_fonts = [f.name for f in fm.fontManager.ttflist]
    for font in korean_fonts:
        if font in available_fonts:
            return FontProperties(family=font)
    return FontProperties(family='sans-serif')

setup_korean_font()

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Libraries imported successfully")
print(f"\n스크리닝 설정:")
print(f"  - 30일 낙폭 기준: {PULLBACK_THRESHOLD_30D}% 이하")
print(f"  - 250일 낙폭 기준: {PULLBACK_THRESHOLD_250D}% 이하")
print(f"  - 종목 크기: {', '.join(COMPANY_SIZES)}")


✓ Korean font set to: AppleGothic
Libraries imported successfully

스크리닝 설정:
  - 30일 낙폭 기준: 70% 이하
  - 250일 낙폭 기준: 70% 이하
  - 종목 크기: 중형주, 소형주


In [11]:
# Connect to DuckDB
db_path = Path.cwd().parent / "data" / "data.duckdb"
conn = duckdb.connect(str(db_path), read_only=True)

print(f"Connected to DuckDB at: {db_path}")
print(f"Database exists: {db_path.exists()}")


Connected to DuckDB at: /Volumes/kgcrom-2tb/kgcrom/workspace/cluefin/data/data.duckdb
Database exists: True


In [12]:
# Database statistics
stats_query = """
SELECT 
    'Stock Metadata' as table_name,
    COUNT(*) as record_count,
    COUNT(DISTINCT stock_code) as unique_stocks
FROM stock_metadata
UNION ALL
SELECT 
    'Daily Charts' as table_name,
    COUNT(*) as record_count,
    COUNT(DISTINCT stock_code) as unique_stocks
FROM daily_charts
"""

stats_df = conn.execute(stats_query).df()
print("\n📊 Database Statistics:\n")
display(stats_df)

# Check company size distribution
size_query = """
SELECT 
    company_size,
    COUNT(*) as stock_count
FROM stock_metadata
WHERE company_size IS NOT NULL
GROUP BY company_size
ORDER BY stock_count DESC
"""

size_df = conn.execute(size_query).df()
print("\n📏 Company Size Distribution:\n")
display(size_df)

# Check stock state patterns
state_query = """
SELECT 
    CASE 
        WHEN stock_state LIKE '%신용가능%' THEN '신용가능'
        ELSE '신용불가능'
    END as credit_status,
    CASE
        WHEN stock_state LIKE '%관리종목%' THEN 'Yes'
        WHEN stock_state LIKE '%거래정지%' THEN 'Yes'
        ELSE 'No'
    END as has_issues,
    COUNT(*) as stock_count
FROM stock_metadata
WHERE stock_state IS NOT NULL
GROUP BY credit_status, has_issues
ORDER BY stock_count DESC
"""

state_df = conn.execute(state_query).df()
print("\n💳 Stock State Distribution:\n")
display(state_df)



📊 Database Statistics:



,table_name,record_count,unique_stocks
0,Stock Metadata,2606,2606
1,Daily Charts,1798617,2606



📏 Company Size Distribution:



,company_size,stock_count
0,소형주,1869
1,중형주,494
2,대형주,199
3,,44



💳 Stock State Distribution:



,credit_status,has_issues,stock_count
0,신용가능,No,1569
1,신용불가능,No,916
2,신용불가능,Yes,118
3,신용가능,Yes,3


## 관심종목 1: 30일 과대 낙폭주 (중/소형주, 신용가능)

**스크리닝 기준:**
- 30일 거래일 중 고가 대비 현재 종가 -70% 이하
- 중형주, 소형주만
- 신용가능 종목만
- 관리종목 및 거래정지 제외
- HC + LC 오름차순 정렬


In [13]:
# 관심종목 1 스크리닝 쿼리 (DuckDB SQL)
watchlist_1_query = f"""
WITH daily_data AS (
    SELECT 
        dc.stock_code,
        dc.date,
        dc.close,
        dc.high,
        dc.low,
        sm.stock_name,
        sm.company_size,
        sm.stock_state,
        sm.market_name,
        sm.industry_name,
        sm.market_cap,
        -- 30일 고점/저점 계산 (window function)
        MAX(dc.high) OVER (
            PARTITION BY dc.stock_code 
            ORDER BY dc.date 
            ROWS BETWEEN {LOOKBACK_DAYS_SHORT - 1} PRECEDING AND CURRENT ROW
        ) as high_period,
        MIN(dc.low) OVER (
            PARTITION BY dc.stock_code 
            ORDER BY dc.date 
            ROWS BETWEEN {LOOKBACK_DAYS_SHORT - 1} PRECEDING AND CURRENT ROW
        ) as low_period,
        ROW_NUMBER() OVER (PARTITION BY dc.stock_code ORDER BY dc.date DESC) as rn
    FROM daily_charts dc
    JOIN stock_metadata sm ON dc.stock_code = sm.stock_code
    WHERE sm.company_size IN ('중형주', '소형주')
      AND sm.stock_state LIKE '%신용가능%'
      AND sm.stock_state NOT LIKE '%관리종목%'
      AND sm.stock_state NOT LIKE '%거래정지%'
),
latest_data AS (
    SELECT 
        stock_code,
        stock_name,
        company_size,
        stock_state,
        market_name,
        industry_name,
        market_cap,
        date,
        close,
        high_period,
        low_period,
        -- HC: 고점 대비 현재가 등락률
        ((close - high_period) / CAST(high_period AS DOUBLE) * 100) as HC,
        -- LC: 저점 대비 현재가 등락률  
        ((close - low_period) / CAST(low_period AS DOUBLE) * 100) as LC
    FROM daily_data
    WHERE rn = 1  -- 최신 날짜만
)
SELECT 
    stock_code,
    stock_name,
    company_size,
    market_name,
    industry_name,
    market_cap,
    date as latest_date,
    close as current_price,
    high_period as high_{LOOKBACK_DAYS_SHORT}d,
    low_period as low_{LOOKBACK_DAYS_SHORT}d,
    ROUND(HC, 2) as HC,
    ROUND(LC, 2) as LC,
    ROUND(HC + LC, 2) as HC_LC_SUM
FROM latest_data
WHERE (high_period / 100 * {PULLBACK_THRESHOLD_30D}) > close  -- 과대 낙폭 기준
ORDER BY HC_LC_SUM ASC
"""

print(f"Executing Watchlist 1 query (30-day pullback, credit available)...")
watchlist_1 = conn.execute(watchlist_1_query).df()

print(f"\n✓ Watchlist 1 screening complete!")
print(f"Total stocks found: {len(watchlist_1):,}")
print(f"\n{'='*120}")
print(f"  관심종목 1: 30일 과대 낙폭주 (중/소형주, 신용가능) - TOP 50")
print(f"{'='*120}\n")
display(watchlist_1.head(50))

# Summary statistics
print(f"\n📊 Summary Statistics:\n")
display(watchlist_1[['HC', 'LC', 'HC_LC_SUM']].describe())


Executing Watchlist 1 query (30-day pullback, credit available)...

✓ Watchlist 1 screening complete!
Total stocks found: 34

  관심종목 1: 30일 과대 낙폭주 (중/소형주, 신용가능) - TOP 50



,stock_code,stock_name,company_size,market_name,industry_name,market_cap,latest_date,current_price,high_30d,low_30d,HC,LC,HC_LC_SUM
0,481070,에이유브랜즈,중형주,코스닥,유통,2118,2025-10-31,14960,26600,14890,-43.76,0.47,-43.29
1,260970,에스앤디,중형주,코스닥,음식료/담배,2132,2025-10-31,73700,134000,72300,-45.00,1.94,-43.06
2,068240,다원시스,중형주,코스닥,전기/전자,1446,2025-10-31,3790,6350,3790,-40.31,0.00,-40.31
3,415380,스튜디오삼익,소형주,코스닥,유통,269,2025-10-31,2350,4215,2250,-44.25,4.44,-39.80
4,469750,아이비젼웍스,소형주,코스닥,기계/장비,451,2025-10-31,1309,2100,1303,-37.67,0.46,-37.21
5,109670,씨싸이트,소형주,코스닥,섬유/의류,370,2025-10-31,6340,10370,6200,-38.86,2.26,-36.60
6,033340,좋은사람들,소형주,코스닥,섬유/의류,1626,2025-10-31,1677,2610,1669,-35.75,0.48,-35.27
7,212710,아이에스티이,소형주,코스닥,전기/전자,903,2025-10-31,9720,15000,9640,-35.20,0.83,-34.37
8,499790,GS피앤엘,중형주,거래소,금융,7605,2025-10-31,38350,59300,37650,-35.33,1.86,-33.47
9,461300,아이스크림미디어,중형주,코스닥,IT 서비스,2341,2025-10-31,17680,27550,17010,-35.83,3.94,-31.89



📊 Summary Statistics:



,HC,LC,HC_LC_SUM
count,34.00,34.00,34.00
mean,-34.17,6.77,-27.40
std,4.08,12.43,13.83
min,-45.00,0.00,-43.29
25%,-35.64,1.92,-33.08
50%,-32.59,3.03,-29.26
75%,-31.41,4.59,-27.37
max,-30.15,62.88,32.55


## 관심종목 2: 250일 과대 낙폭주 (중/소형주, 신용가능)

**스크리닝 기준:**
- 250일 거래일 중 고가 대비 현재 종가 -70% 이하
- 중형주, 소형주만
- 신용가능 종목만
- 관리종목 및 거래정지 제외
- HC + LC 오름차순 정렬


In [14]:
# 관심종목 2 스크리닝 쿼리 (DuckDB SQL)
watchlist_2_query = f"""
WITH daily_data AS (
    SELECT 
        dc.stock_code,
        dc.date,
        dc.close,
        dc.high,
        dc.low,
        sm.stock_name,
        sm.company_size,
        sm.stock_state,
        sm.market_name,
        sm.industry_name,
        sm.market_cap,
        -- 250일 고점/저점 계산 (window function)
        MAX(dc.high) OVER (
            PARTITION BY dc.stock_code 
            ORDER BY dc.date 
            ROWS BETWEEN {LOOKBACK_DAYS_LONG - 1} PRECEDING AND CURRENT ROW
        ) as high_period,
        MIN(dc.low) OVER (
            PARTITION BY dc.stock_code 
            ORDER BY dc.date 
            ROWS BETWEEN {LOOKBACK_DAYS_LONG - 1} PRECEDING AND CURRENT ROW
        ) as low_period,
        ROW_NUMBER() OVER (PARTITION BY dc.stock_code ORDER BY dc.date DESC) as rn
    FROM daily_charts dc
    JOIN stock_metadata sm ON dc.stock_code = sm.stock_code
    WHERE sm.company_size IN ('중형주', '소형주')
      AND sm.stock_state LIKE '%신용가능%'
      AND sm.stock_state NOT LIKE '%관리종목%'
      AND sm.stock_state NOT LIKE '%거래정지%'
),
latest_data AS (
    SELECT 
        stock_code,
        stock_name,
        company_size,
        stock_state,
        market_name,
        industry_name,
        market_cap,
        date,
        close,
        high_period,
        low_period,
        -- HC: 고점 대비 현재가 등락률
        ((close - high_period) / CAST(high_period AS DOUBLE) * 100) as HC,
        -- LC: 저점 대비 현재가 등락률  
        ((close - low_period) / CAST(low_period AS DOUBLE) * 100) as LC
    FROM daily_data
    WHERE rn = 1  -- 최신 날짜만
)
SELECT 
    stock_code,
    stock_name,
    company_size,
    market_name,
    industry_name,
    market_cap,
    date as latest_date,
    close as current_price,
    high_period as high_{LOOKBACK_DAYS_LONG}d,
    low_period as low_{LOOKBACK_DAYS_LONG}d,
    ROUND(HC, 2) as HC,
    ROUND(LC, 2) as LC,
    ROUND(HC + LC, 2) as HC_LC_SUM
FROM latest_data
WHERE (high_period / 100 * {PULLBACK_THRESHOLD_250D}) > close  -- 과대 낙폭 기준
ORDER BY HC_LC_SUM ASC
"""

print(f"Executing Watchlist 2 query (250-day pullback, credit available)...")
watchlist_2 = conn.execute(watchlist_2_query).df()

print(f"\n✓ Watchlist 2 screening complete!")
print(f"Total stocks found: {len(watchlist_2):,}")
print(f"\n{'='*120}")
print(f"  관심종목 2: 250일 과대 낙폭주 (중/소형주, 신용가능) - TOP 50")
print(f"{'='*120}\n")
display(watchlist_2.head(50))

# Summary statistics
print(f"\n📊 Summary Statistics:\n")
display(watchlist_2[['HC', 'LC', 'HC_LC_SUM']].describe())


Executing Watchlist 2 query (250-day pullback, credit available)...

✓ Watchlist 2 screening complete!
Total stocks found: 505

  관심종목 2: 250일 과대 낙폭주 (중/소형주, 신용가능) - TOP 50



,stock_code,stock_name,company_size,market_name,industry_name,market_cap,latest_date,current_price,high_250d,low_250d,HC,LC,HC_LC_SUM
0,045660,에이텍,소형주,코스닥,전기/전자,919,2025-10-31,11120,46300,10900,-75.98,2.02,-73.96
1,025950,동신건설,소형주,코스닥,건설,1285,2025-10-31,15300,79100,14010,-80.66,9.21,-71.45
2,479960,위너스,소형주,코스닥,전기/전자,793,2025-10-31,11570,39400,11500,-70.63,0.61,-70.03
3,224110,에이텍모빌리티,소형주,코스닥,전기/전자,522,2025-10-31,9780,32500,9720,-69.91,0.62,-69.29
4,209640,와이제이링크,소형주,코스닥,기계/장비,845,2025-10-31,2970,9695,2955,-69.37,0.51,-68.86
5,407400,꿈비,소형주,코스닥,기타제조,726,2025-10-31,4960,15870,4955,-68.75,0.10,-68.65
6,480370,씨케이솔루션,소형주,거래소,건설,987,2025-10-31,9030,28000,9020,-67.75,0.11,-67.64
7,068240,다원시스,중형주,코스닥,전기/전자,1446,2025-10-31,3790,11490,3790,-67.01,0.00,-67.01
8,091120,이엠텍,소형주,코스닥,전기/전자,1599,2025-10-31,9330,30700,9060,-69.61,2.98,-66.63
9,465480,인스피언,소형주,코스닥,IT 서비스,583,2025-10-31,5750,20000,5440,-71.25,5.70,-65.55



📊 Summary Statistics:



,HC,LC,HC_LC_SUM
count,505.00,505.00,505.00
mean,-42.09,31.38,-10.71
std,10.78,38.86,41.69
min,-90.00,0.00,-73.96
25%,-46.97,5.07,-35.82
50%,-38.86,18.84,-21.36
75%,-34.07,42.36,2.69
max,-30.01,306.52,263.19


## 관심종목 3: 250일 과대 낙폭주 (중/소형주, 신용불가능)

**스크리닝 기준:**
- 250일 거래일 중 고가 대비 현재 종가 -70% 이하
- 중형주, 소형주만
- **신용불가능 종목만** (관심 1, 2와의 차이점)
- 관리종목 및 거래정지 제외
- HC + LC 오름차순 정렬


In [15]:
# 관심종목 3 스크리닝 쿼리 (DuckDB SQL)
watchlist_3_query = f"""
WITH daily_data AS (
    SELECT 
        dc.stock_code,
        dc.date,
        dc.close,
        dc.high,
        dc.low,
        sm.stock_name,
        sm.company_size,
        sm.stock_state,
        sm.market_name,
        sm.industry_name,
        sm.market_cap,
        -- 250일 고점/저점 계산 (window function)
        MAX(dc.high) OVER (
            PARTITION BY dc.stock_code 
            ORDER BY dc.date 
            ROWS BETWEEN {LOOKBACK_DAYS_LONG - 1} PRECEDING AND CURRENT ROW
        ) as high_period,
        MIN(dc.low) OVER (
            PARTITION BY dc.stock_code 
            ORDER BY dc.date 
            ROWS BETWEEN {LOOKBACK_DAYS_LONG - 1} PRECEDING AND CURRENT ROW
        ) as low_period,
        ROW_NUMBER() OVER (PARTITION BY dc.stock_code ORDER BY dc.date DESC) as rn
    FROM daily_charts dc
    JOIN stock_metadata sm ON dc.stock_code = sm.stock_code
    WHERE sm.company_size IN ('중형주', '소형주')
      AND sm.stock_state NOT LIKE '%신용가능%'  -- 신용불가능 종목만
      AND sm.stock_state NOT LIKE '%관리종목%'
      AND sm.stock_state NOT LIKE '%거래정지%'
),
latest_data AS (
    SELECT 
        stock_code,
        stock_name,
        company_size,
        stock_state,
        market_name,
        industry_name,
        market_cap,
        date,
        close,
        high_period,
        low_period,
        -- HC: 고점 대비 현재가 등락률
        ((close - high_period) / CAST(high_period AS DOUBLE) * 100) as HC,
        -- LC: 저점 대비 현재가 등락률  
        ((close - low_period) / CAST(low_period AS DOUBLE) * 100) as LC
    FROM daily_data
    WHERE rn = 1  -- 최신 날짜만
)
SELECT 
    stock_code,
    stock_name,
    company_size,
    market_name,
    industry_name,
    market_cap,
    date as latest_date,
    close as current_price,
    high_period as high_{LOOKBACK_DAYS_LONG}d,
    low_period as low_{LOOKBACK_DAYS_LONG}d,
    ROUND(HC, 2) as HC,
    ROUND(LC, 2) as LC,
    ROUND(HC + LC, 2) as HC_LC_SUM
FROM latest_data
WHERE (high_period / 100 * {PULLBACK_THRESHOLD_250D}) > close  -- 과대 낙폭 기준
ORDER BY HC_LC_SUM ASC
"""

print(f"Executing Watchlist 3 query (250-day pullback, NO credit)...")
watchlist_3 = conn.execute(watchlist_3_query).df()

print(f"\n✓ Watchlist 3 screening complete!")
print(f"Total stocks found: {len(watchlist_3):,}")
print(f"\n{'='*120}")
print(f"  관심종목 3: 250일 과대 낙폭주 (중/소형주, 신용불가능) - TOP 50")
print(f"{'='*120}\n")
display(watchlist_3.head(50))

# Summary statistics
print(f"\n📊 Summary Statistics:\n")
display(watchlist_3[['HC', 'LC', 'HC_LC_SUM']].describe())


Executing Watchlist 3 query (250-day pullback, NO credit)...

✓ Watchlist 3 screening complete!
Total stocks found: 463

  관심종목 3: 250일 과대 낙폭주 (중/소형주, 신용불가능) - TOP 50



,stock_code,stock_name,company_size,market_name,industry_name,market_cap,latest_date,current_price,high_250d,low_250d,HC,LC,HC_LC_SUM
0,166480,코아스템켐온,소형주,코스닥,제약,509,2025-10-31,1550,13347,1541,-88.39,0.58,-87.80
1,214610,롤링스톤,소형주,코스닥,의료/정밀기기,298,2025-10-31,3550,22692,3505,-84.36,1.28,-83.07
2,019490,엑시큐어하이트론,소형주,거래소,전기/전자,479,2025-10-31,627,3965,586,-84.19,7.00,-77.19
3,321820,아티스트컴퍼니,소형주,코스닥,IT 서비스,787,2025-10-31,5050,23000,4950,-78.04,2.02,-76.02
4,457600,벡트,소형주,코스닥,전기/전자,347,2025-10-31,2530,10640,2510,-76.22,0.80,-75.42
5,313760,캐리,소형주,코스닥,전기/전자,206,2025-10-31,1837,7770,1810,-76.36,1.49,-74.87
6,121850,코이즈,소형주,코스닥,화학,186,2025-10-31,606,2500,598,-75.76,1.34,-74.42
7,200350,아티스트스튜디오,소형주,코스닥,오락/문화,741,2025-10-31,7410,28150,7400,-73.68,0.14,-73.54
8,009410,태영건설,중형주,거래소,건설,4675,2025-10-31,1571,6110,1534,-74.29,2.41,-71.88
9,065060,지엔코,소형주,코스닥,섬유/의류,191,2025-10-31,801,2742,797,-70.79,0.50,-70.29



📊 Summary Statistics:



,HC,LC,HC_LC_SUM
count,463.00,463.00,463.00
mean,-47.25,32.03,-15.22
std,12.74,53.41,55.43
min,-88.39,0.00,-87.80
25%,-54.64,4.82,-42.30
50%,-44.40,15.02,-29.26
75%,-37.34,37.97,-4.48
max,-30.11,601.59,545.95


## 관심종목 비교 분석

각 관심종목의 특성과 분포를 비교합니다.


In [16]:
# 관심종목 비교 통계
comparison_data = {
    '관심종목': ['관심 1 (30일, 신용가능)', '관심 2 (250일, 신용가능)', '관심 3 (250일, 신용불가능)'],
    '종목 수': [len(watchlist_1), len(watchlist_2), len(watchlist_3)],
    'HC 평균': [watchlist_1['HC'].mean() if len(watchlist_1) > 0 else 0,
               watchlist_2['HC'].mean() if len(watchlist_2) > 0 else 0,
               watchlist_3['HC'].mean() if len(watchlist_3) > 0 else 0],
    'LC 평균': [watchlist_1['LC'].mean() if len(watchlist_1) > 0 else 0,
               watchlist_2['LC'].mean() if len(watchlist_2) > 0 else 0,
               watchlist_3['LC'].mean() if len(watchlist_3) > 0 else 0],
    'HC+LC 평균': [watchlist_1['HC_LC_SUM'].mean() if len(watchlist_1) > 0 else 0,
                  watchlist_2['HC_LC_SUM'].mean() if len(watchlist_2) > 0 else 0,
                  watchlist_3['HC_LC_SUM'].mean() if len(watchlist_3) > 0 else 0]
}

comparison_df = pd.DataFrame(comparison_data)
print("\n📊 관심종목 비교 요약:\n")
display(comparison_df)

# 업종별 분포 (관심종목 2 기준)
if len(watchlist_2) > 0:
    print("\n🏭 업종별 분포 (관심 2 기준):\n")
    industry_dist = watchlist_2['industry_name'].value_counts().head(10)
    display(industry_dist)

# 시장별 분포 비교
market_comparison = pd.DataFrame({
    '관심 1': watchlist_1['market_name'].value_counts() if len(watchlist_1) > 0 else pd.Series(),
    '관심 2': watchlist_2['market_name'].value_counts() if len(watchlist_2) > 0 else pd.Series(),
    '관심 3': watchlist_3['market_name'].value_counts() if len(watchlist_3) > 0 else pd.Series()
}).fillna(0).astype(int)

print("\n📈 시장별 분포:\n")
display(market_comparison)



📊 관심종목 비교 요약:



,관심종목,종목 수,HC 평균,LC 평균,HC+LC 평균
0,"관심 1 (30일, 신용가능)",34,-34.17,6.77,-27.40
1,"관심 2 (250일, 신용가능)",505,-42.09,31.38,-10.71
2,"관심 3 (250일, 신용불가능)",463,-47.25,32.03,-15.22



🏭 업종별 분포 (관심 2 기준):



industry_name
IT 서비스     73
전기/전자      58
화학         49
기계/장비      38
제약         33
유통         29
금속         29
일반서비스      27
의료/정밀기기    26
운송장비/부품    22
Name: count, dtype: int64


📈 시장별 분포:



,관심 1,관심 2,관심 3
market_name,,,
코스닥,26,370,395
거래소,8,135,68


## CSV 파일로 내보내기 (선택 사항)

관심종목 리스트를 CSV 파일로 저장합니다.


In [ ]:
# CSV 파일로 저장 (주석 해제하여 사용)
export_path = Path.cwd().parent / "data"

# 관심종목 1 저장
# watchlist_1_path = export_path / "watchlist_1_30d_credit.csv"
# watchlist_1.to_csv(watchlist_1_path, index=False, encoding='utf-8-sig')
# print(f"✓ Watchlist 1 exported to: {watchlist_1_path}")

# 관심종목 2 저장
# watchlist_2_path = export_path / "watchlist_2_250d_credit.csv"
# watchlist_2.to_csv(watchlist_2_path, index=False, encoding='utf-8-sig')
# print(f"✓ Watchlist 2 exported to: {watchlist_2_path}")

# 관심종목 3 저장
# watchlist_3_path = export_path / "watchlist_3_250d_no_credit.csv"
# watchlist_3.to_csv(watchlist_3_path, index=False, encoding='utf-8-sig')
# print(f"✓ Watchlist 3 exported to: {watchlist_3_path}")

print("CSV export is commented out. Uncomment the lines above to save files.")


In [ ]:
# Close database connection
conn.close()
print("\n✓ Database connection closed")
print("\n" + "="*100)
print("  과대 낙폭주 스크리닝 완료!")
print("="*100)
print(f"\n스크리닝 결과:")
print(f"  - 관심 1 (30일, 신용가능): {len(watchlist_1):,}개 종목")
print(f"  - 관심 2 (250일, 신용가능): {len(watchlist_2):,}개 종목")
print(f"  - 관심 3 (250일, 신용불가능): {len(watchlist_3):,}개 종목")
print(f"\n이 결과는 투자 권유가 아니며, 교육 및 연구 목적으로만 사용하세요.")
